Install Kaggle Package



In [2]:
!pip install -q kaggle

 Mounts Google Drive to the Colab environment, making files accessible

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create directory for the Kaggle API configuration file.


In [3]:
!mkdir -p /root/.kaggle

Copies Kaggle API key (kaggle.json) from Google Drive to the .kaggle directory

In [4]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

Set Permissions

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

Downloads the dataset from Kaggle using the dataset identifier

In [6]:
!kaggle datasets download -d danushkumarv/indian-monuments-image-dataset

Dataset URL: https://www.kaggle.com/datasets/danushkumarv/indian-monuments-image-dataset
License(s): CC-BY-NC-SA-4.0
 98% 609M/621M [00:09<00:00, 78.8MB/s]
100% 621M/621M [00:09<00:00, 65.2MB/s]


Extracts the downloaded ZIP file into the ('/content') directory

In [7]:
import zipfile

with zipfile.ZipFile('indian-monuments-image-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [8]:
#import the libraries

import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

#Function to Load Data

def get_data_from_folder(path):
  class_names = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

  IMG_SIZE = 224
  data = []
  for class_name in class_names:
    try:
      files = glob.glob(os.path.join(path, class_name, "*"))
      for f in files:
        img = cv2.imread(f)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        data.append([np.array(img), class_names.index(class_name)])
    except:
      pass
  np.random.shuffle(data)
  return data, class_names

#Load Training and Testing Data

training_data, training_class_names = get_data_from_folder('/content/Indian-monuments/images/train')
testing_data, testing_class_names = get_data_from_folder('/content/Indian-monuments/images/test')

print(training_class_names)  # This will print the extracted monument types for training data



['Khajuraho', 'tanjavur temple', 'qutub_minar', 'basilica_of_bom_jesus', 'Gateway of India', 'Chhota_Imambara', 'victoria memorial', 'tajmahal', 'jamali_kamali_tomb', 'Humayun_s Tomb', 'Fatehpur Sikri', 'charminar', 'Ellora Caves', 'iron_pillar', 'Ajanta Caves', 'golden temple', 'mysore_palace', 'Sun Temple Konark', 'alai_darwaza', 'lotus_temple', 'Charar-E- Sharif', 'India gate pics', 'alai_minar', 'hawa mahal pics']


In [9]:
#Image Preprocessing Function

import numpy as np
def preprocess_image(img):

  img = img / 255.0


  img = np.expand_dims(img, axis=0)

  return img

#Preprocess Training and Testing Data

for i, (img, label) in enumerate(training_data):
  training_data[i][0] = preprocess_image(img)

for i, (img, label) in enumerate(testing_data):
  testing_data[i][0] = preprocess_image(img)


In [10]:
#Import OneHotEncoder

from sklearn.preprocessing import OneHotEncoder

#Separate and Encode Labels

training_labels = np.array([label for _, label in training_data])
testing_labels = np.array([label for _, label in testing_data])


encoder = OneHotEncoder(sparse=False)
encoder.fit(training_labels.reshape(-1, 1))


training_labels = encoder.transform(training_labels.reshape(-1, 1))
testing_labels = encoder.transform(testing_labels.reshape(-1, 1))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image size
IMG_SIZE = 224

# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(training_class_names), activation='softmax')
])

# Compile the model
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Data generators for efficient batch processing
datagen = ImageDataGenerator(rescale=0.2)

# Replace 'path_to_train_data' and 'path_to_validation_data' with the actual paths
train_generator = datagen.flow_from_directory(
    '/content/Indian-monuments/images/train', # Update with the correct path
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    '/content/Indian-monuments/images/test', # Update with the correct path
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical'
)

# Train the model
cnn_model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# Evaluate the model
cnn_model.evaluate(validation_generator)

Found 3666 images belonging to 24 classes.
Found 1049 images belonging to 24 classes.
Epoch 1/5
115/115 [==============================] - 366s 3s/step - loss: 43.6087 - accuracy: 0.0747 - val_loss: 3.1300 - val_accuracy: 0.1049
Epoch 2/5
115/115 [==============================] - 367s 3s/step - loss: 2.9286 - accuracy: 0.1626 - val_loss: 2.8972 - val_accuracy: 0.2231
Epoch 3/5
115/115 [==============================] - 376s 3s/step - loss: 2.3064 - accuracy: 0.3622 - val_loss: 2.9672 - val_accuracy: 0.3108
Epoch 4/5
115/115 [==============================] - 366s 3s/step - loss: 1.6054 - accuracy: 0.5363 - val_loss: 3.0107 - val_accuracy: 0.3622
Epoch 5/5
33/33 [==============================] - 31s 929ms/step - loss: 3.7222 - accuracy: 0.3870


[3.7222142219543457, 0.3870352804660797]

In [15]:
# Predictive system
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = preprocess_image(img)
    prediction = cnn_model.predict(img)
    predicted_class = training_class_names[np.argmax(prediction)]
    return predicted_class

# Example usage:
image_path = '/content/Indian-monuments/images/test/Ajanta Caves/1.jpg'  # Update with an actual image path
predicted_class = predict_image(image_path)
print(f'The predicted class is: {predicted_class}')

1/1 [==============================] - 0s 50ms/step
The predicted class is: mysore_palace
